Decision Tree from scratch

In [1]:
#importing libraries 
import numpy as np
import pandas as pd 
import math

Data processing

In [2]:
col_names = ['edibility','cap-shape', "cap-surface", "cap-color", "bruises", 
             "odor", "gill-attachment", "gill-spacing", "gill-size", "gill-color", 
             "stalk-shape","stalk-root", "stalk-surface-above-ring", "stalk-surface-below-ring",
               "stalk-color-above-ring", "stalk-color-below-ring", "veil-type",
                "veil-color", "ring-number", "ring-type", "spore-print-color",
                 "population", "habitat" ]

#read data
mushroom_df = pd.read_table('agaricus-lepiota.data', sep=',', header = None, names = col_names)

#number of rows
print("num of row: " , len(mushroom_df))
#number of columns
print("num of col: ", len(mushroom_df.columns))

#check the index number where "?" exist 
missing_rows_num = []
for col in mushroom_df:
    count = -1
    for rows in mushroom_df[col]:
        count += 1
        if rows == "?":
            missing_rows_num.append(count)

missing_rows_num2 = np.unique(missing_rows_num)

#remove missing value rows
mushroom_df2 = mushroom_df
for i in missing_rows_num2:
    mushroom_df2 = mushroom_df2.drop(i)

#num of rows after remove all missing rows
print("num of row after removing: ", len(mushroom_df2)), 

#split a dataset into 70% for train, 30% for test (pandas.sample is used)
train_df =mushroom_df2.sample(frac=0.7,random_state=123)
test_df =mushroom_df2.drop(train_df.index)


Ycol_name = "edibility"


num of row:  8124
num of col:  23
num of row after removing:  5644


Implement DecisionTree algorithm with a train procedure - using information gain criterion.

In [3]:
#calculating entropy
def calc_entropy(train_df, col_name):
    Y_vale_list = np.unique(train_df[Ycol_name]) 
    total_entropy = 0
    
    for i in Y_vale_list:
        num_of_i_row = len(train_df[train_df[col_name] == i])
        #entropy of i
        if num_of_i_row != 0:
            i_Entropy = ((num_of_i_row/len(train_df)) * math.log2(num_of_i_row/len(train_df)))
            total_entropy += i_Entropy
    return -total_entropy


#calculate IG value
def calc_IG (train_df, attribute_name, Ycol_name):
    #list of values in the feature 
    value_in_attribute = np.unique(train_df[attribute_name])
    total_X_entropy = 0

    for i in value_in_attribute:
        #split df to sub df which only contains specific value
        splited_df = train_df[train_df[attribute_name] == i]
        i_value_entropy = calc_entropy(splited_df, Ycol_name)
        i_entropy = (len(splited_df)/len(train_df)) * i_value_entropy
        total_X_entropy += i_entropy

    Y_entropy = calc_entropy(train_df, Ycol_name)
    #return IG value
    return (Y_entropy - total_X_entropy)


def find_highest_IG(train_df, Ycol_name):
    #minimum IG value is 0
    max_IG = -1
    best_node = "error"
    Xs_df = train_df.drop(labels=Ycol_name, axis = 1)
    #compare each attributes' IG value 
    for attribute in Xs_df:
        IG_value = calc_IG(train_df, attribute, Ycol_name)
        if max_IG < IG_value:
            max_IG = IG_value
            best_node = attribute
    return best_node


def decision_tree(train_df, Ycol_name, tree = None):
    #find (root) node
    node = find_highest_IG(train_df, Ycol_name)
    value_in_attribute = np.unique(train_df[node])
    
    #creating dic to print tree
    if tree is None:
        tree= {}
        tree[node] = {}

    #build a tree
    for i in value_in_attribute:
        splited_df = train_df[train_df[node]== i]
        impurity_check = np.unique(splited_df[Ycol_name],return_counts = True)
        #check the splited node has pure leaf or not
        if len(impurity_check[0]) == 1:
            leaf_node = impurity_check[0][0]
            tree[node][i] = leaf_node
        else:
            tree[node][i] = decision_tree(splited_df, Ycol_name)
    return tree



decision_tree(train_df, Ycol_name)

{'odor': {'a': 'e',
  'c': 'p',
  'f': 'p',
  'l': 'e',
  'm': 'p',
  'n': {'spore-print-color': {'k': 'e',
    'n': 'e',
    'r': 'p',
    'w': {'cap-color': {'c': 'e',
      'g': 'e',
      'n': 'e',
      'p': 'e',
      'w': 'p',
      'y': 'p'}}}},
  'p': 'p'}}

Implement decisoin tree depth control

In [5]:

def stopping_depth(depth):
    depth_number = depth
    count = 0
    return decision_tree2(train_df, Ycol_name, depth_number, count)

def decision_tree2(train_df, Ycol_name, depth_number, count, tree = None):
    #find (root) node
    node = find_highest_IG(train_df, Ycol_name)
    value_in_attribute = np.unique(train_df[node])
    
    #count the depth
    if tree is None:
        count += 1
        tree= {}
        tree[node] = {}

    for i in value_in_attribute:
        splited_df = train_df[train_df[node]== i]
        impurity_check = np.unique(splited_df[Ycol_name],return_counts = True)
        
        #number of e and p
        each_value_num = impurity_check[1].tolist()
        #values of edibility 
        each_value = impurity_check[0].tolist()
        
        #check the splited node has pure leaf or not
        if len(impurity_check[0]) == 1:
            leaf_node = impurity_check[0][0]
            tree[node][i] = leaf_node
        elif count == depth_number:
            #majority rule applied to choose the leaf node
            if each_value_num[1] < each_value_num[0]:
                tree[node][i] = each_value[0]
            else:
                tree[node][i] = each_value[1]
        else:
            tree[node][i] = decision_tree2(splited_df, Ycol_name, depth_number, count)
    return tree

#tree which stopped at depth 2
stopping_depth(2)


{'odor': {'a': 'e',
  'c': 'p',
  'f': 'p',
  'l': 'e',
  'm': 'p',
  'n': {'spore-print-color': {'k': 'e', 'n': 'e', 'r': 'p', 'w': 'e'}},
  'p': 'p'}}

In [14]:
#tree which stopped at depth 3
stopping_depth(3)

{'odor': {'a': 'e',
  'c': 'p',
  'f': 'p',
  'l': 'e',
  'm': 'p',
  'n': {'spore-print-color': {'k': 'e',
    'n': 'e',
    'r': 'p',
    'w': {'cap-color': {'c': 'e',
      'g': 'e',
      'n': 'e',
      'p': 'e',
      'w': 'p',
      'y': 'p'}}}},
  'p': 'p'}}

In [15]:
#tree which stopped at depth 4
stopping_depth(4)


{'odor': {'a': 'e',
  'c': 'p',
  'f': 'p',
  'l': 'e',
  'm': 'p',
  'n': {'spore-print-color': {'k': 'e',
    'n': 'e',
    'r': 'p',
    'w': {'cap-color': {'c': 'e',
      'g': 'e',
      'n': 'e',
      'p': 'e',
      'w': 'p',
      'y': 'p'}}}},
  'p': 'p'}}

Implement a test procedure that takes new data and the trained model and returns a prediction

In [16]:
def check_TP(tree, splitted_df, Ycol_name):
    key_list = list(tree.keys())
    node = key_list[0]
    node_value = splitted_df[node][0]
    predict_Y = tree[node][node_value]
    actual_Y = splitted_df[Ycol_name][0]
    if predict_Y == actual_Y:
        return 1
    elif type(predict_Y) is not dict:
        return 0
    else:
        return check_TP(predict_Y, splitted_df, Ycol_name)
    
    
def test(tree, test_df, Ycol_name):
    TP= 0
    for row in test_df.to_numpy():
        splitted_df = pd.DataFrame([row])
        splitted_df.columns = col_names
        #result is 1 when it is TP, 0 otherwise
        result = check_TP(tree, splitted_df, Ycol_name)
        TP += result
    #calculate accuracy percentage
    return (TP/(len(test_df))) * 100

tree = decision_tree(train_df, Ycol_name)
test(tree, test_df, Ycol_name)
    

100.0

**test evaluation:** I used accuracy to test my decision tree model performance. I calculate the number of correct (TP) values. Based on the number of TP in the testing is used in the accuracy formula: TP/total. The test(tree, test_df, Ycol_name) function results with 100% accuracy, which means my model predicts well with the mushroom data frame. 

Explain whether the evaluation method can indicate whether your tree is over- or underfitting

Overfitting is when a model has 100% accuracy with the train data while it has poor accuracy with the test data. This indicates that our model did not capture the relationship which are in test data. In this case, we can prune the tree to make it more general. On the other hand, underfitting is when a model has poor accuracy with both the train and test data. This indicates that our model fails to capture relationship which are in both data sets. In this case, we can increase the depth of the tree in order to increase the accuracy of both data sets. When my training decision tree only has depth 2, 

In [17]:
#accuracy when tree depth is 2
test(stopping_depth(2), test_df, Ycol_name)

99.88186650915534

it has a 99.9% accuracy with the test data.  
This indicates that my evaluation method to generate a tree is not underfitting and perfectly fits. However, in general, if a trained model has 100% accuracy with the train data, it would have lower accuracy since the tree is overfitting. 